### 1. 路径操作
- Path() 等价于 Path('.') 表示当前工作目录
- .absolute()与.resolve()实现将相对路径解析为绝对路径
- .parent和os.path.dirname()都实现返回父目录,而前者是对象后者是字符串，前者更现代，但是系统模块，需要以字符串为输入

In [1]:
import os, sys, pathlib
sys.path.insert(0, os.path.dirname(pathlib.Path().absolute()))

In [3]:
# Import libraries
import random
import os
import numpy as np 
import pandas as pd 
import requests
import pandas_datareader as web

# Date
from datetime import date, timedelta, datetime

# EDA
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import ydata_profiling as pp

# FE
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.inspection import permutation_importance
import eli5
from eli5.sklearn import PermutationImportance
import shap

# Time Series - EDA and Modelling
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Modeling and preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from prophet import Prophet
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

In [6]:
cryptocurrency = 'BTC'
date_start = datetime(2020, 4, 1)
date_end = datetime(2025, 6, 16)
# date_end = dt.datetime.now()

### 2. Download Data
- pandas_datareader (数据--FRED, IEX, Alpha Vantage 等)，但是用不了，被反爬了
- 只好用yfinance调用yahoo的数据
- Adj Close 的调整主要是为了股票这种会有分红、拆股的资产设计的，加密货币没有这些因素，所以没区别

In [8]:
import yfinance as yf
import datetime as dt

def get_data(cryptocurrency, date_start, date_end=None):
    if date_end is None:
        date_end = dt.datetime.now()

    df = yf.download(f'{cryptocurrency}-USD', start=date_start, end=date_end)
    return df

df = get_data(cryptocurrency, date_start, date_end)


[*********************100%***********************]  1 of 1 completed


In [9]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2020-04-01,6606.776367,6612.573730,6202.373535,6437.319336,40346426266
2020-04-02,6793.624512,7088.247559,6595.918457,6606.776367,47660646124
2020-04-03,6733.387207,7003.220703,6673.335938,6797.396484,38976504903
2020-04-04,6867.527344,6878.953613,6696.484863,6738.382812,33185988584
2020-04-05,6791.129395,6883.414062,6715.929199,6862.537598,29510409856


In [8]:
df.tail()

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2025-06-11,108686.625000,110384.218750,108086.328125,110261.796875,50842662052
2025-06-12,105929.054688,108780.695312,105785.687500,108685.914062,54843867968
2025-06-13,106090.968750,106182.546875,102822.023438,105924.593750,69550440846
2025-06-14,105472.406250,106203.757812,104379.367188,106108.085938,38007870453
2025-06-15,105552.023438,106157.101562,104519.882812,105464.843750,36744307742


In [7]:
df.corr()

,Price,Close,High,Low,Open,Volume
,Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Price,Ticker,,,,,
Close,BTC-USD,1.000000,0.999306,0.999127,0.998376,0.338759
High,BTC-USD,0.999306,1.000000,0.998639,0.999222,0.351902
Low,BTC-USD,0.999127,0.998639,1.000000,0.998925,0.320376
Open,BTC-USD,0.998376,0.999222,0.998925,1.000000,0.339146
Volume,BTC-USD,0.338759,0.351902,0.320376,0.339146,1.000000


In [11]:
df.describe()

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
count,1993.000000,1993.000000,1993.000000,1993.000000,1.993000e+03
mean,41221.706258,42045.188944,40265.254860,41174.547101,3.405234e+10
std,26083.335826,26550.294433,25534.127246,26055.727476,1.983974e+10
min,4970.788086,5331.833984,4106.980957,5002.578125,5.331173e+09
25%,20569.919922,20988.394531,20087.134766,20553.371094,2.096570e+10
50%,36575.140625,37480.187500,34940.824219,36491.789062,3.055876e+10
75%,58719.484375,59847.359375,57217.824219,58706.847656,4.165678e+10
max,111673.281250,111970.171875,109285.070312,111679.359375,3.509679e+11


In [9]:
df.corr()['Close']

,Ticker,BTC-USD
Price,Ticker,
Close,BTC-USD,1.000000
High,BTC-USD,0.999306
Low,BTC-USD,0.999127
Open,BTC-USD,0.998376
Volume,BTC-USD,0.338759


In [14]:
df.columns

MultiIndex([( 'Close', 'BTC-USD'),
            (  'High', 'BTC-USD'),
            (   'Low', 'BTC-USD'),
            (  'Open', 'BTC-USD'),
            ('Volume', 'BTC-USD')],
           names=['Price', 'Ticker'])

### 3. EDA
- 3.1 Market Cap
- 

In [14]:
from IPython.display import display

In [17]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2020-04-01,6606.776367,6612.573730,6202.373535,6437.319336,40346426266
2020-04-02,6793.624512,7088.247559,6595.918457,6606.776367,47660646124
2020-04-03,6733.387207,7003.220703,6673.335938,6797.396484,38976504903
2020-04-04,6867.527344,6878.953613,6696.484863,6738.382812,33185988584
2020-04-05,6791.129395,6883.414062,6715.929199,6862.537598,29510409856


In [11]:
crypto = pd.Series(df.market_cap.head(20).tolist(), index=df.name.head(20).tolist(), name="BTC")
crypto.plot.pie(figsize=(10, 10))

AttributeError: 'DataFrame' object has no attribute 'market_cap'

In [6]:
import requests

def get_my_ip():
    try:
        ip = requests.get('https://api.ipify.org').text
        print("Current public IP:", ip)
    except Exception as e:
        print("Error:", e)

get_my_ip()


Current public IP: 140.245.49.112
